# Plot M cross section according to ocean sigma grid

In [1]:
%matplotlib widget

In [10]:
import pandas as pd
import hvplot.xarray
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import holoviews as hv
from holoviews import dim, opts
from JulesD3D.plot2D import makeVerticalLengthSlice
from JulesD3D.utils import ncFilePicker, quickDF, folderPicker, easyTimeFormat
# hv.extension('matplotlib')

In [11]:
parent_folder = '/Users/julesblom/ThesisResults/'
# parent_folder = '/Volumes/Seagate Expansion Drive/FinalResults'
scenario_folder = folderPicker(parent_folder)
scenario_folder

Select(description='Scenario folder:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom…

In [12]:
nc_filename = ncFilePicker(scenario_folder.value)
nc_filename

Select(description='NetCDF file:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom/The…

In [14]:
trim = xr.open_dataset(nc_filename.value)

In [18]:
density_vertical_plot = makeVerticalLengthSlice(trim, 'RHO')

In [19]:
plot_settings = {
    'height': 400,
    'width': 800,
    'rasterize': True,
    'dynamic': True,
    'grid': True,
    'legend': True,
    'attr_labels': True,    
}
# trim.keys()

In [22]:
density_vertical_plot.isel(M=100).hvplot.quadmesh('N_KMAXOUT_RESTR', 'depth_center', 
                        groupby=['time'],
                        cmap='viridis',
                        clim=(1030,1050),
                        clabel=trim['RHO'].attrs['units'],
                        ylim=(-700,-250),
                        **plot_settings
                       )

IndexError: index 180 is out of bounds for axis 0 with size 132

In [6]:
# depth = trim.SIG_INTF @ trim.DPS # depth at interfaces
# # Reshape to have time as first dimension
# depth = depth.transpose('time', 'M', 'N', 'SIG_INTF')
# depth = depth.assign_attrs({"unit":"m", "long_name": "Depth at Sigma-layer interfaces"})
# print(depth.dims)
# print(depth.shape)

In [8]:
# depth_center = trim.SIG_LYR @ trim.DPS
# # Reshape to have time as first dimension
# depth_center = depth_center.transpose('time', 'M', 'N', 'SIG_LYR', transpose_coords=False)
# depth_center = depth_center.assign_attrs({"unit":"m", "long_name": "Depth at Sigma-layer centers"})
# print(depth_center.dims)
# print(depth_center.shape)

('time', 'M', 'N', 'SIG_LYR')
(12, 132, 182, 80)


In [9]:
# # Add depth and depth_center coords to DataSet
# # trim.coords['depth'] = depth
# trim.coords['depth_center'] = depth_center
# trim.RHO.coords

Coordinates:
    XZ             (M, N) float64 ...
    YZ             (M, N) float64 ...
  * KMAXOUT_RESTR  (KMAXOUT_RESTR) int32 0 1 2 3 4 5 6 ... 73 74 75 76 77 78 79
  * time           (time) datetime64[ns] 2020-02-01T20:30:00 ... 2020-02-02T16:18:00

In [35]:
# density_m = trim['RHO'].assign_coords(depth_center=
#     (('time', 'M', 'N', 'KMAXOUT_RESTR'),depth_center.values)
# )

In [36]:
# density_m

<xarray.DataArray 'RHO' (time: 12, KMAXOUT_RESTR: 80, M: 132, N: 182)>
[23063040 values with dtype=float32]
Coordinates:
    XZ               (M, N) float64 ...
    YZ               (M, N) float64 ...
  * KMAXOUT_RESTR    (KMAXOUT_RESTR) int32 0 1 2 3 4 5 6 ... 74 75 76 77 78 79
  * time             (time) datetime64[ns] 2020-02-01T20:30:00 ... 2020-02-02T16:18:00
  * M                (M) int64 0 1 2 3 4 5 6 7 ... 125 126 127 128 129 130 131
    M_KMAXOUT_RESTR  (M, KMAXOUT_RESTR) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    depth_center     (time, M, N, KMAXOUT_RESTR) float32 0.0 0.0 0.0 ... 0.0 0.0
Dimensions without coordinates: N
Attributes:
    long_name:  Density per layer in zeta point
    units:      kg/m3
    grid:       grid
    location:   face

# hvPlot vertical cross M-section

In [37]:
# _, mesh_M_intf = np.meshgrid(trim.SIG_INTF.values, trim.XZ[:,0])
# _, mesh_M_lyr = np.meshgrid(trim.SIG_LYR.values, trim.XZ[:,0])

In [38]:
print(mesh_M_intf.shape)
print(mesh_M_lyr.shape)

(132, 81)
(132, 80)


In [39]:
# Unknown bug?
# M_KMAXOUT = xr.DataArray(mesh_M_intf, dims=['M', 'KMAXOUT'], 
#                          coords={'M':trim.M, 'KMAXOUT': trim.KMAXOUT},
#                          attrs={'units':'m', 'long_name': 'X-SIG_INTF Meshgrid'})

In [40]:
M_KMAXOUT_RESTR = xr.DataArray(mesh_M_lyr, dims=['M', 'KMAXOUT_RESTR'], 
                               coords={'M': trim.M, 'KMAXOUT_RESTR': trim.KMAXOUT_RESTR},
                               attrs={'units':'m', 'long_name': 'X-SIG_LYR Meshgrid'})

In [41]:
# Add NKMAXOUT and NKMAXOUT_RESTR coords to DataSet
# trim.coords["M_KMAXOUT"] = M_KMAXOUT
trim.coords["M_KMAXOUT_RESTR"] = M_KMAXOUT_RESTR

In [42]:
trim['RHO'].isel(N=55)

<xarray.DataArray 'RHO' (time: 12, KMAXOUT_RESTR: 80, M: 132)>
[126720 values with dtype=float32]
Coordinates:
    XZ               (M) float64 ...
    YZ               (M) float64 ...
  * KMAXOUT_RESTR    (KMAXOUT_RESTR) int32 0 1 2 3 4 5 6 ... 74 75 76 77 78 79
  * time             (time) datetime64[ns] 2020-02-01T20:30:00 ... 2020-02-02T16:18:00
  * M                (M) int64 0 1 2 3 4 5 6 7 ... 125 126 127 128 129 130 131
    M_KMAXOUT_RESTR  (M, KMAXOUT_RESTR) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    long_name:  Density per layer in zeta point
    units:      kg/m3
    grid:       grid
    location:   face

In [43]:
trim['RHO'].isel(N=50).hvplot.quadmesh('M_KMAXOUT_RESTR', 'depth_center', 
                        groupby='time',
                        rasterize=True,
                        grid=False,
                        cmap='viridis',
                        width=700, height=400,
                        colorbar=True,
                        legend=True,
                        ylim=(-700,0),
                        clabel=trim.LYRFRAC.attrs['units'], 
#                         clim=(1025,1025.1),
                        attr_labels=True,
                        dynamic=True,
                       )

Invoked as dynamic_operation(numpy.datetime64('2020-02-01T20:30:00.000000000'))
Invoked as dynamic_operation(numpy.datetime64('2020-02-01T20:30:00.000000000'), height=400, scale=1.0, width=700, x_range=None, y_range=(-700, 0))
Invoked as dynamic_operation(numpy.datetime64('2020-02-01T20:30:00.000000000'), height=400, scale=1.0, width=700, x_range=None, y_range=(-700, 0))


DataError: xarray Dataset must define coordinates for all defined kdims, [Dimension('depth_center')] coordinates not found.

XArrayInterface expects gridded data, for more information on supported datatypes see http://holoviews.org/user_guide/Gridded_Datasets.html

:DynamicMap   [time]